# Elasticsearch

## Mục lục:
- [Elasticsearch](#elasticsearch)
  - [0. Thiết lập môi trường & kết nối Elasticsearch](#0-thiết-lập-môi-trường-kết-nối-elasticsearch)
  - [1. Indexing](#1-indexing)
    - [1.1. BiomedCLIP embedding pipeline](#11-biomedclip-embedding-pipeline)
    - [1.2. Khai báo index & ingest dữ liệu](#12-khai-báo-index-ingest-dữ-liệu)
      - [1.2.1 Khai báo index](#121-khai-báo-index)
      - [1.2.2. Hàm ingest dữ liệu](#122-hàm-ingest-dữ-liệu)
      - [1.2.3 Kiểm tra thống kê index](#123-kiểm-tra-thống-kê-index)
    - [1.3. Inverted index, segments và cơ chế refresh (near-real-time)](#13-inverted-index-segments-và-cơ-chế-refresh-near-real-time)
      - [1.3.1. Inverted index là gì?](#131-inverted-index-là-gì)
      - [1.3.2. Segments](#132-segments)
      - [1.3.3. Refresh vs flush và near-real-time search](#133-refresh-vs-flush-và-near-real-time-search)
      - [Refresh (mặc định ~1s)](#refresh-mặc-định-1s)
      - [Flush](#flush)
      - [1.3.4. Demo near-real-time với `radiology_text`](#134-demo-near-real-time-với-radiology_text)
  - [2. Query Processing](#2-query-processing)
    - [2.1. Full-text queries trên `radiology_text`](#21-full-text-queries-trên-radiology_text)
      - [2.1.1. Term & match](#211-term-match)
      - [2.1.2. Match_phrase](#212-match_phrase)
    - [2.2. Bool query với filter](#22-bool-query-với-filter)
    - [2.3. Highlight & sort](#23-highlight-sort)
    - [2.4 Aggregations trên metadata (terms & date_histogram)](#24-aggregations-trên-metadata-terms-date_histogram)
    - [2.5 Profile queries – nhìn bên trong engine](#25-profile-queries-nhìn-bên-trong-engine)
    - [2.6 Script score](#26-script-score)
    - [2.7 Vector kNN search trên `image_vector`](#27-vector-knn-search-trên-image_vector)
    - [2.8 Semantic search trên `clinicians_notes`](#28-semantic-search-trên-clinicians_notes)
  - [3. Transaction & Concurrency Control](#3-transaction-concurrency-control)
    - [3.1 Transaction model: single-document atomicity](#31-transaction-model-single-document-atomicity)
    - [3.2 Translog, refresh & flush](#32-translog-refresh-flush)
    - [3.3 Bulk & `update_by_query`](#33-bulk-update_by_query)
    - [3.4 Optimistic concurrency control (OCC)](#34-optimistic-concurrency-control-occ)
  - [4. Comparison with Other Databases](#4-comparison-with-other-databases)
    - [4.1. So sánh kỹ thuật tổng quan](#41-so-sánh-kỹ-thuật-tổng-quan)
    - [4.2. Schema PostgreSQL cho benchmark](#42-schema-postgresql-cho-benchmark)
    - [4.3. Ingest to PostgreSQL](#43-ingest-to-postgresql)
      - [4.3.1. Ingest text data](#431-ingest-text-data)
      - [4.3.2. Ingest image data](#432-ingest-image-data)
    - [4.4. Workloads benchmark](#44-workloads-benchmark)
      - [4.4.1. **Top-k text search**](#441-top-k-text-search)
      - [4.4.2. **Tag filter + aggregation**](#442-tag-filter-aggregation)
      - [4.4.3. **Image vector kNN**](#443-image-vector-knn)
    - [4.5 Kết luận benchmark (định tính)](#45-kết-luận-benchmark-định-tính)
    - [4.6 Vai trò đề xuất](#46-vai-trò-đề-xuất)

## 0. Thiết lập môi trường & kết nối Elasticsearch <a id="section-0"></a>

Thiết lập notebook cho bộ MRI (DICOM + Clinician's Notes):

- Quản lý thư viện khoa học (`numpy`, `pandas`, `matplotlib`).
- Đọc DICOM, tạo embedding và tqdm để theo dõi ingest.
- Kết nối Elasticsearch cục bộ và khai báo đường dẫn dữ liệu.


In [13]:
# Comprehensive imports for all parts
import os
import sys
import json
import time
from time import sleep
import threading
from pprint import pprint
import urllib3

# Scientific libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm

# DICOM and ML libraries
import pydicom
import torch
from PIL import Image
from sentence_transformers import SentenceTransformer

# For BiomedCLIP
try:
    from open_clip import create_model_from_pretrained
except ImportError:
    print('Note: open_clip not installed, some image embedding features may not work')

# Elasticsearch
from elasticsearch import Elasticsearch, helpers, ConflictError
from elasticsearch.helpers import scan

# PostgreSQL
import psycopg2
from psycopg2.extras import execute_batch

# Configuration constants
ES_URL = os.getenv("ES_URL", "http://localhost:9200")
TEXT_INDEX = "radiology_text"
VECTOR_INDEX = "radiology_vectors"

# ROOT_DIR = "../../mri_images"
# REPORT_XLSX = "../../Radiologists Report.xlsx"

ROOT_DIR = "../../data/images"
REPORT_XLSX = "../../data/text/Radiologists Report.xlsx"

PG_HOST = os.getenv("PG_HOST", "localhost")
PG_PORT = int(os.getenv("PG_PORT", "5432"))
PG_DB   = os.getenv("PG_DB",   "database")
PG_USER = os.getenv("PG_USER", "username")
PG_PASSWORD = os.getenv("PG_PASSWORD", "password")

# Connect to Elasticsearch
es = Elasticsearch(ES_URL)
try:
    print(f"Connected to Elasticsearch {es.info().body['version']['number']}")
except:
    print("Could not connect to Elasticsearch - check if it's running")


Connected to Elasticsearch 9.2.1


## 1. Indexing <a id="section-1"></a>
### 1.1. BiomedCLIP embedding pipeline <a id="section-1-1"></a>

BiomedCLIP được dùng để tạo vector đặc trưng (512 chiều) cho từng lát MRI, bao gồm:

- Tự động nhận `cuda`/`cpu`.
- Chuẩn hoá ảnh xám → PIL → tiền xử lý của OpenCLIP.
- Hàm tiện ích:
  - `get_image_vector_from_array`
  - `get_image_vector_from_dicom`
  - `_infer_dims` để suy ra số chiều.


In [14]:
# ================================
# MODEL INIT
# ================================
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using device:", device)

# Load BiomedCLIP từ HF Hub
model, preprocess = create_model_from_pretrained(
    'hf-hub:microsoft/BiomedCLIP-PubMedBERT_256-vit_base_patch16_224'
)
model.to(device).eval()
print("BiomedCLIP loaded.")


def _pil_from_grayscale_array(img: np.ndarray) -> Image.Image:
    """
    Chuẩn hoá 2D grayscale numpy (float/uint16/…) -> PIL uint8 [0,255].
    """
    img = img.astype(np.float32)
    img_min = img.min()
    img -= img_min
    img_max = img.max()
    if img_max > 0:
        img /= img_max
    img_uint8 = (img * 255).astype(np.uint8)
    return Image.fromarray(img_uint8)


@torch.inference_mode()
def get_image_vector_from_array(img: np.ndarray):
    """
    Convert 2D numpy array (grayscale) -> BiomedCLIP embedding (L2-normalized).
    Return: list[float]
    """
    pil_img = _pil_from_grayscale_array(img)
    image_input = preprocess(pil_img).unsqueeze(0).to(device)
    image_features = model.encode_image(image_input)
    # L2 normalize để cosine similarity meaningful
    image_features /= image_features.norm(dim=-1, keepdim=True)
    return image_features[0].cpu().numpy().tolist()


@torch.inference_mode()
def get_image_vector_from_dicom(dicom_path: str):
    """
    Đọc file DICOM (*.dcm, *.ima), lấy pixel_array -> embed bằng BiomedCLIP.
    """
    ds = pydicom.dcmread(dicom_path)
    img = ds.pixel_array.astype(np.float32)
    return get_image_vector_from_array(img)


@torch.inference_mode()
def _infer_dims() -> int:
    dummy = Image.fromarray(np.zeros((224, 224), dtype=np.uint8))
    image_input = preprocess(dummy).unsqueeze(0).to(device)
    feats = model.encode_image(image_input)
    return int(feats.shape[-1])

VECTOR_DIMS = _infer_dims()
print(f"VECTOR_DIMS = {VECTOR_DIMS}")

Using device: cuda


/home/tram/dbms/DBMS-Assignment/src/.venv/lib/python3.13/site-packages/huggingface_hub/file_download.py:942: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


BiomedCLIP loaded.
VECTOR_DIMS = 512


### 1.2. Khai báo index & ingest dữ liệu <a id="section-1-2"></a>
#### 1.2.1 Khai báo index <a id="section-1-2-1"></a>
Ta dùng hai index:

- **`radiology_text`**
  - Lưu 1 document / bệnh nhân.
  - Trường `clinicians_notes` là `text` với custom analyzer:
    - tokenizer: `standard`.
    - filter: `lowercase`, `asciifolding`, `stop`, `porter_stem`.
  - Trường `patient_id` là `keyword` với `normalizer` để chuẩn hoá (lowercase + bỏ dấu).

- **`radiology_vectors`**
  - Lưu metadata từng lát cắt MRI + vector ảnh:
    - `patient_id`, `patient_sex`, `patient_age`, `body_part_examined`,
      `sequence_name`, `modality` → `keyword` + `normalizer`.
    - `tags` → `keyword` list để filter nhanh.
    - `study_date` → `date`.
    - `study_description`, `series_description` → `text`.
    - Tham số kỹ thuật (`magnetic_field_strength`, `slice_thickness`,
      `echo_time`, `repetition_time`) → `float`.
    - `slice_index` → `integer`.
    - `image_vector` → `dense_vector` với `dims = VECTOR_DIMS`, `similarity = "cosine"`.
    - `image_path` → `keyword`.

In [15]:
def recreate_index(name, body):
    """Xoá index (nếu tồn tại) rồi tạo lại từ đầu."""
    if es.indices.exists(index=name):
        es.indices.delete(index=name)
    es.indices.create(index=name, body=body)
    print(f"Recreated index: {name}")

# --- Text index: clinicians_notes ---
text_index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "analysis": {
            "analyzer": {
                "custom_text_analyzer": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": [
                        "lowercase",
                        "asciifolding",
                        "stop",
                        "porter_stem",
                    ],
                }
            },
            "normalizer": {
                "lowercase_normalizer": {
                    "type": "custom",
                    "filter": ["lowercase", "asciifolding"],
                }
            },
        },
    },
    "mappings": {
        "properties": {
            "patient_id": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "clinicians_notes": {
                "type": "text",
                "analyzer": "custom_text_analyzer",
            },
        }
    },
}

# --- Vector index: image_vector + metadata ---
vector_index_body = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0,
        "index.refresh_interval": "5s",
        "analysis": {
            "normalizer": {
                "lowercase_normalizer": {
                    "type": "custom",
                    "filter": ["lowercase", "asciifolding"],
                }
            }
        },
    },
    "mappings": {
        "properties": {
            "patient_id": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "patient_sex": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "patient_age": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "body_part_examined": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "sequence_name": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "modality": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "tags": {
                "type": "keyword",
                "normalizer": "lowercase_normalizer",
            },
            "study_date": {
                "type": "date",
                "format": "yyyyMMdd||yyyy-MM-dd||strict_date_optional_time",
            },
            "study_description": {"type": "text"},
            "series_description": {"type": "text"},
            "magnetic_field_strength": {"type": "float"},
            "slice_thickness": {"type": "float"},
            "echo_time": {"type": "float"},
            "repetition_time": {"type": "float"},
            "slice_index": {"type": "integer"},
            "image_vector": {
                "type": "dense_vector",
                "dims": VECTOR_DIMS,
                "index": True,
                "similarity": "cosine",
            },
            "image_path": {"type": "keyword"},
        }
    },
}

recreate_index(TEXT_INDEX, text_index_body)
recreate_index(VECTOR_INDEX, vector_index_body)

Recreated index: radiology_text
Recreated index: radiology_vectors


#### 1.2.2. Hàm ingest dữ liệu <a id="section-1-2-2"></a>

Bao gồm:

1. `extract_dicom_metadata(ds)` chuẩn hoá metadata + gán `tags` (body part, sequence, modality…).
2. `ingest_text_reports()` đọc file Excel và bulk vào `radiology_text` (1 doc / patient).
3. `ingest_all()` duyệt thư mục DICOM, chọn lát SAG, embed bằng BiomedCLIP và bulk vào `radiology_vectors`.
4. `print_index_stats()` để theo dõi `_cat/indices`, `_stats`, `_segments`.

In [16]:
def extract_dicom_metadata(ds):
    def safe(tag, default=None):
        return getattr(ds, tag, default)

    def to_float(x):
        try:
            return float(x)
        except Exception:
            return 0.0

    meta = {
        "patient_id": str(safe("PatientID", "")),
        "patient_sex": str(safe("PatientSex", "")),
        "patient_age": str(safe("PatientAge", "")),
        "body_part_examined": str(safe("BodyPartExamined", "")),
        "study_description": str(safe("StudyDescription", "")),
        "series_description": str(safe("SeriesDescription", "")),
        "sequence_name": str(safe("SequenceName", "")),
        "magnetic_field_strength": to_float(safe("MagneticFieldStrength", 0)),
        "slice_thickness": to_float(safe("SliceThickness", 0)),
        "echo_time": to_float(safe("EchoTime", 0)),
        "repetition_time": to_float(safe("RepetitionTime", 0)),
    }

    meta["modality"] = str(safe("Modality", ""))

    study_date = safe("StudyDate", "")
    if study_date:
        meta["study_date"] = str(study_date)

    tags = []
    bp = meta.get("body_part_examined")
    if bp:
        tags.append(bp)

    sd = meta.get("study_description", "")
    if sd:
        parts = (
            str(sd)
            .lower()
            .replace("^", " ")
            .split()
        )
        parts = [p.strip() for p in parts if p.strip()]
        tags.extend(parts)

    seq = meta.get("sequence_name", "")
    if seq:
        tags.append(seq)

    modality = meta.get("modality")
    if modality:
        tags.append(modality)

    if tags:
        meta["tags"] = tags

    return meta

In [17]:
def ingest_text_reports():
    df = pd.read_excel(REPORT_XLSX)

    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("'", "", regex=False)
    )

    required = {"patient_id", "clinicians_notes"}
    missing = required - set(df.columns)
    if missing:
        raise ValueError(f"Missing required columns after normalization: {missing}")

    df = df.dropna(subset=["patient_id", "clinicians_notes"])
    df["patient_id"] = df["patient_id"].astype(str).str.strip()

    print(f"Total text records to ingest: {len(df)}")

    actions = []
    for _, row in df.iterrows():
        pid = row["patient_id"]
        doc = {
            "patient_id": pid,
            "clinicians_notes": str(row["clinicians_notes"]),
        }
        actions.append({
            "_index": TEXT_INDEX,
            "_id": pid,
            "_source": doc,
        })

    if actions:
        helpers.bulk(es, actions)
        es.indices.refresh(index=TEXT_INDEX)

    print(f"Ingested {len(actions)} text docs into {TEXT_INDEX}")

In [18]:
def ingest_all():
    actions_vector = []
    SLICE_RANGE = 3  # ±3 lát quanh lát giữa → tổng cộng 7 lát/sequence

    for pid in tqdm(sorted(os.listdir(ROOT_DIR)), desc="Patients"):
        patient_path = os.path.join(ROOT_DIR, pid)
        if not os.path.isdir(patient_path):
            continue

        for study in os.listdir(patient_path):
            study_path = os.path.join(patient_path, study)
            if not os.path.isdir(study_path):
                continue

            for seq in os.listdir(study_path):
                seq_path = os.path.join(study_path, seq)
                if not os.path.isdir(seq_path):
                    continue

                if "SAG" not in seq.upper():
                    continue

                ima_files = [f for f in os.listdir(seq_path) if f.endswith(".ima")]
                if not ima_files:
                    continue

                ima_files.sort()
                mid = len(ima_files) // 2
                slice_indices = range(
                    max(0, mid - SLICE_RANGE),
                    min(len(ima_files), mid + SLICE_RANGE + 1),
                )

                for idx in slice_indices:
                    dicom_path = os.path.join(seq_path, ima_files[idx])
                    try:
                        ds = pydicom.dcmread(dicom_path)
                        img = ds.pixel_array.astype(np.float32)

                        img -= img.min()
                        if img.max() > 0:
                            img /= img.max()

                        meta = extract_dicom_metadata(ds)
                        vec = get_image_vector_from_array(img)

                        meta.update(
                            {
                                "slice_index": idx,
                                "image_vector": vec,
                                "image_path": dicom_path,
                            }
                        )

                        actions_vector.append(
                            {
                                "_index": VECTOR_INDEX,
                                "_id": f"{pid}_{seq}_{idx}",
                                "_source": meta,
                            }
                        )

                    except Exception as e:
                        print(f" Error reading {dicom_path}: {e}")

    print("\nBulk indexing VECTOR docs...")
    if actions_vector:
        helpers.bulk(es, actions_vector)
        es.indices.refresh(index=VECTOR_INDEX)
    print(f"Done! Indexed {len(actions_vector)} slices\n")

In [19]:
def print_index_stats():
    print("\n=== _cat/indices ===")
    try:
        print(es.cat.indices(index=f"{TEXT_INDEX},{VECTOR_INDEX}", v=True))
    except Exception as e:
        print(f"cat.indices not available: {e}")

    print("\n=== Text index stats ===")
    stats_text = es.indices.stats(index=TEXT_INDEX)
    prim_text = stats_text.get("indices", {}).get(TEXT_INDEX, {}).get("primaries", {})
    docs_text = prim_text.get("docs", {}).get("count", 0)
    store_text = prim_text.get("store", {}).get("size_in_bytes", 0)
    print(f"  {TEXT_INDEX}: docs={docs_text}, store={store_text} bytes")

    print("\n=== Vector index stats ===")
    stats_vec = es.indices.stats(index=VECTOR_INDEX)
    prim_vec = stats_vec.get("indices", {}).get(VECTOR_INDEX, {}).get("primaries", {})
    docs_vec = prim_vec.get("docs", {}).get("count", 0)
    store_vec = prim_vec.get("store", {}).get("size_in_bytes", 0)
    print(f"  {VECTOR_INDEX}: docs={docs_vec}, store={store_vec} bytes")

    print("\n=== Segments (vector index) ===")
    seg = es.indices.segments(index=VECTOR_INDEX)

    index_seg = (
        seg.get("indices", {}).get(VECTOR_INDEX)
        or seg.get(VECTOR_INDEX)
    )

    if not index_seg:
        print(f"  {VECTOR_INDEX}: no segments info (index may be empty or API changed)")
        return

    shards = index_seg.get("shards", {})
    if "0" in shards and shards["0"]:
        seg_info = shards["0"][0]
        num_seg = seg_info.get("num_search_segments") or len(seg_info.get("segments", {}))
        print(f"  {VECTOR_INDEX}: num_search_segments={num_seg}")
    else:
        print(f"  {VECTOR_INDEX}: no active shard segments yet")

#### 1.2.3 Kiểm tra thống kê index <a id="section-1-2-3"></a>

Quy trình:

1. Xem stats ban đầu (index vừa tạo, chưa có dữ liệu).
2. Ingest text reports vào `radiology_text`.
3. Ingest ảnh + vector vào `radiology_vectors`.
4. Xem lại `_cat/indices`, `_stats`, `_segments` để so sánh trước/sau.

In [22]:
print("=== BEFORE INGEST ===")
print_index_stats()

print("\n=== INGEST TEXT REPORTS ===")
ingest_text_reports()

print("\n=== INGEST MRI SLICES + VECTORS ===")
ingest_all()

print("\n=== AFTER INGEST ===")
print_index_stats()

=== BEFORE INGEST ===

=== _cat/indices ===
health status index             uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
green  open   radiology_vectors QEvJ4mhuQQeyOGYqrwFhvA   1   0          0            0       249b           249b         249b
green  open   radiology_text    s3jURyzZQFuOblyr2gecyA   1   0        515            0    168.7kb        168.7kb      168.7kb


=== Text index stats ===
  radiology_text: docs=515, store=172844 bytes

=== Vector index stats ===
  radiology_vectors: docs=0, store=249 bytes

=== Segments (vector index) ===
  radiology_vectors: num_search_segments=0

=== INGEST TEXT REPORTS ===
Total text records to ingest: 515
Ingested 515 text docs into radiology_text

=== INGEST MRI SLICES + VECTORS ===


Patients: 100%|██████████| 516/516 [02:46<00:00,  3.10it/s]



Bulk indexing VECTOR docs...
Done! Indexed 8517 slices


=== AFTER INGEST ===

=== _cat/indices ===
health status index             uuid                   pri rep docs.count docs.deleted store.size pri.store.size dataset.size
green  open   radiology_vectors QEvJ4mhuQQeyOGYqrwFhvA   1   0       8363          154      2.5mb          2.5mb        2.5mb
green  open   radiology_text    s3jURyzZQFuOblyr2gecyA   1   0        515           15     96.9kb         96.9kb       96.9kb


=== Text index stats ===
  radiology_text: docs=515, store=99232 bytes

=== Vector index stats ===
  radiology_vectors: docs=8363, store=2707082 bytes

=== Segments (vector index) ===
  radiology_vectors: num_search_segments=4


### 1.3. Inverted index, segments và cơ chế refresh (near-real-time) <a id="section-1-3"></a>

Để hiểu vì sao Elasticsearch vừa search nhanh, vừa gần như *real-time*, cần nắm ba khái niệm:
- **Inverted index** – cấu trúc dữ liệu cho full-text search.
- **Segments** – đơn vị lưu trữ bất biến (immutable) bên trong mỗi shard.
- **Refresh** – cơ chế giúp dữ liệu mới “lộ diện” với search.

#### 1.3.1. Inverted index là gì? <a id="section-1-3-1"></a>
Thay vì lưu dạng "tài liệu → danh sách từ", Elasticsearch dùng **inverted index**: "từ khoá → danh sách tài liệu chứa từ đó".

- Truy vấn `match` sẽ được analyzer xử lý giống như lúc ingest (lowercase, stopwords, stemming...).
- Kết quả trả về nhanh vì chỉ thao tác trên postings list thay vì scan toàn bộ 575 ghi chú.
- Ví dụ nhanh dưới đây dùng chính file Excel Radiologists Report.


In [ ]:
df = pd.read_excel(REPORT_XLSX)
print(f"Dataset shape: {df.shape}")
df.head()


| Term           | Danh sách tài liệu (postings list) |
|----------------|-------------------------------------|
| `l4`           | (Patient 1)                         |
| `5`            | (Patient 1)                         |
| `degenerative` | (Patient 1)                         |
| `annular`      | (Patient 1)                         |
| `disc`         | (Patient 1), (Patient 2), (Patient 3) |
| `bulge`        | (Patient 1)                         |
| `herniation`   | (Patient 2)                         |
| `lss`          | (Patient 3)                         |
| `mri`          | (Patient 3)                         |
| `muscle`       | (Patient 3)                         |
| `spasm`        | (Patient 3)                         |
| `protrusion`   | (Patient 3)                         |

**Quy trình xử lý truy vấn**:
1. Analyzer cắt từ: ví dụ `lss`, `mri`, `muscle`, `spasm`.
2. Elasticsearch tra postings list tương ứng.
3. Gộp & xếp hạng dựa trên BM25 / độ dài tài liệu / tần suất term.
4. Trả về Clinician's Notes của những bệnh nhân chứa các term trên.


#### 1.3.2. Segments <a id="section-1-3-2"></a>

- Mỗi shard được chia thành nhiều **segment** chỉ-đọc, mỗi segment có inverted index riêng.
- Khi ingest batch MRI:
  1. Tài liệu mới ghi vào buffer + translog.
  2. Chu kỳ refresh (~1s) ghi xuống đĩa → tạo `segment_n` mới.
  3. Segment mới được mở cho search, sau đó Elasticsearch merge các segment nhỏ để tối ưu.
- Với 575 bản ghi text + hàng nghìn lát MRI, chỉ sau vài lần merge số segment sẽ ổn định → query đọc tất cả segment đang sống và hợp nhất kết quả.


#### 1.3.3. Refresh vs flush và near-real-time search <a id="section-1-3-3"></a>

Elasticsearch là hệ thống **near-real-time (NRT)**:
- Ngay sau khi index, tài liệu **chưa chắc đã search thấy ngay**.
- Tài liệu chỉ được nhìn thấy trong search **sau lần refresh kế tiếp**. 

Hai quá trình chính trên shard:

#### Refresh (mặc định ~1s)
- Đọc dữ liệu từ buffer, tạo segment mới.
- Mở segment đó cho search.
- Gây ra độ trễ ~1s giữa thời điểm index và lúc search thấy document.

#### Flush
- Ghi dữ liệu bền vững hơn, xoá bớt translog cũ.
- Quan trọng cho durability, **không trực tiếp liên quan** tới việc NRT search có thấy doc mới hay không.


#### 1.3.4. Demo near-real-time với `radiology_text` <a id="section-1-3-4"></a>

Kịch bản demo:

1. Đọc `refresh_interval` hiện tại của `radiology_text`.
2. Tạm thời đặt `refresh_interval = -1` để **tắt auto-refresh**, dễ quan sát.
3. Index một document mới.
4. Search ngay lập tức (trước khi refresh).
5. Gọi `indices.refresh()`.
6. Search lại.
7. Khôi phục `refresh_interval` về giá trị ban đầu.


In [ ]:
settings_before = es.indices.get_settings(index=TEXT_INDEX)
old_refresh = settings_before[TEXT_INDEX]["settings"]["index"].get("refresh_interval", "1s")

print(f"refresh_interval hiện tại của `{TEXT_INDEX}`: {old_refresh}")
es.indices.put_settings(
    index=TEXT_INDEX,
    settings={
        "index": {
            "refresh_interval": "-1"
        }
    }
)
print(f"Đã tạm thời đặt refresh_interval = -1 cho `{TEXT_INDEX}`.")


In [ ]:
doc = {
    "patient_id": "demo_nrt",
    "clinicians_notes": "LSS MRI shows regression of previous disc bulge."
}

index_resp = es.index(index=TEXT_INDEX, document=doc)
doc_id = index_resp.get("_id")
print(f"Indexed new document with _id = {doc_id}")

query = {
    "match": {
        "clinicians_notes": "regression of previous disc bulge"
    }
}

print("\n=== Search BEFORE refresh ===")
resp_before = es.search(index=TEXT_INDEX, query=query)
total_before = resp_before["hits"]["total"]["value"]
print("Total hits:", total_before)
for hit in resp_before["hits"]["hits"]:
    print(" - patient_id =", hit["_source"].get("patient_id"))


In [ ]:
es.indices.refresh(index=TEXT_INDEX)
print("\nĐã gọi es.indices.refresh()")

print("\n=== Search AFTER refresh ===")
resp_after = es.search(index=TEXT_INDEX, query=query)
total_after = resp_after["hits"]["total"]["value"]
print("Total hits:", total_after)
for hit in resp_after["hits"]["hits"]:
    print(" - patient_id =", hit["_source"].get("patient_id"))

es.indices.put_settings(
    index=TEXT_INDEX,
    settings={
        "index": {
            "refresh_interval": old_refresh
        }
    }
)
print(f"\nĐã khôi phục refresh_interval của `{TEXT_INDEX}` về: {old_refresh}")


## 2. Query Processing <a id="section-2"></a>

In [ ]:
def print_text_hits(resp, max_hits: int = 5) -> None:
    hits = resp["hits"]["hits"]
    total = resp["hits"]["total"]["value"]
    print(f"Total hits: {total}")
    print("-" * 60)
    for h in hits[:max_hits]:
        src = h["_source"]
        pid = src.get("patient_id")
        score = h.get("_score")
        note = src.get("clinicians_notes", "")
        if len(note) > 220:
            note = note[:220] + "..."
        print(f"patient_id = {pid}, _score = {score}")
        print("  clinicians_notes:", note)
        print("-" * 60)


def print_vector_hits(resp, max_hits: int = 5) -> None:
    hits = resp["hits"]["hits"]
    total = resp["hits"]["total"]["value"]
    print(f"Total hits: {total}")
    print("-" * 60)
    for h in hits[:max_hits]:
        src = h["_source"]
        print(
            f"patient_id={src.get('patient_id')}, "
            f"body_part={src.get('body_part_examined')}, "
            f"slice_index={src.get('slice_index')}, "
            f"_score={h.get('_score')}"
        )
        print("  image_path:", src.get("image_path"))
        print("-" * 60)


### 2.1. Full-text queries trên `radiology_text` <a id="section-2-1"></a>
#### 2.1.1. Term & match <a id="secion-2-1-1"></a>
- **`match`**:
  - Phân tích (analyze) query string → token.
  - Áp dụng cùng analyzer với field.
  - Trả về tài liệu có *bất kỳ token* trùng khớp, kèm scoring (BM25).

- **`term`**:
  - **Không** phân tích query string.
  - So khớp đúng **term** trong inverted index.
  - Hợp với `keyword` / ID / mã, không hợp với full-text có analyzer.

Ta sẽ demo:

- `match` vs `term` trên cùng field để thấy sự khác biệt.
- `match_phrase`: yêu cầu cụm từ liên tiếp.

In [ ]:
## match
match_query = {
    "match": {
        "clinicians_notes": "neural canals"
    }
}

## term
term_query = {
    "term": {
        "clinicians_notes": "Neural"
    }
}

print("=== MATCH: \"neural canals\" ===")
resp_match = es.search(index=TEXT_INDEX, query=match_query)
print_text_hits(resp_match)

print("\n=== TERM: clinicians_notes = 'Neural' (no analysis) ===")
resp_term = es.search(index=TEXT_INDEX, query=term_query)
print_text_hits(resp_term)

#### 2.1.2. Match_phrase <a id="section-2-1-2"></a>
`match_phrase` yêu cầu các term xuất hiện **liên tục** theo đúng thứ tự trong field.

Ví dụ: `"neural canals"` sẽ chỉ match khi notes chứa cụm này (sau khi phân tích với analyzer).

In [ ]:
match_phrase_query = {
    "match_phrase": {
        "clinicians_notes": "neural canals"
    }
}

resp_phrase = es.search(index=TEXT_INDEX, query=match_phrase_query)
print("=== MATCH_PHRASE: \"neural canals\" ===")
print_text_hits(resp_phrase)

### 2.2. Bool query với filter <a id="section-2-2"></a>
Cấu trúc `bool`:

- `must` / `should` / `must_not`: **scored** clauses (ảnh hưởng `_score`).
- `filter`: điều kiện **không tính điểm**, chỉ dùng để include/exclude:
  - Kết quả filter có thể được cache.
  - Hợp cho `term`, `range`, `exists`, v.v.

In [ ]:
## Retrive all clinicians notes contains both "diffuse" and "compressing" word from the first 1000 patients
bool_filter_query = {
    "bool": {
        "must": [
            { "match": { "clinicians_notes": "diffuse"} },
            { "match": {"clinicians_notes": "compressing" }}
        ],
        "filter": [
            {"term": {"patient_id": "100"}}
        ]   
    }
}

resp_bool = es.search(index=TEXT_INDEX, query=bool_filter_query)
print("=== BOOL: must match 'diffuse' & 'compressing' + filter patient_id=500 ===")
print_text_hits(resp_bool)

### 2.3. Highlight & sort <a id="section-2-3"></a>
Highlight giúp:

- Hiển thị đoạn text có match query.
- Bọc các term khớp trong `<em>...</em>` (mặc định).

Sorting:

- Mặc định: `_score` desc.
- Có thể sort theo field khác (vd `patient_id`), nhưng nhớ:
  - sort theo field `keyword` hoặc `numeric` sẽ bỏ `_score` nếu không thêm `_score` trong sort list.

In [ ]:
highlight_query = {
    "query": {
        "match": {
            "clinicians_notes": "muscle spasm"
        }
    },
    "highlight": {
        "fields": {
            "clinicians_notes": {}
        }
    }
}

response = es.search(index=TEXT_INDEX, body=highlight_query)
print_text_hits(response)


### 2.4 Aggregations trên metadata (terms & date_histogram) <a id="section-2-4"></a>
Các field metadata trong `radiology_vectors`:

- `body_part_examined` (keyword, normalized).
- `tags` (keyword list).
- `study_date` (date: `yyyyMMdd`).

Ta sẽ:

1. Đếm số lát cắt theo `body_part_examined` bằng `terms` agg.
2. Đếm theo thời gian (`study_date`) bằng `date_histogram` (theo tháng).

In [ ]:
agg_body = {
    "size": 0,
    "aggs": {
        "by_body_part": {
            "terms": {
                "field": "body_part_examined",
                "size": 10
            }
        }
    }
}

resp_agg_body = es.search(index=VECTOR_INDEX, body=agg_body)
buckets = resp_agg_body["aggregations"]["by_body_part"]["buckets"]
print("=== Số lát cắt theo body_part_examined ===")
for b in buckets:
    print(f"{b['key']}: {b['doc_count']} slices")


# 2) Date histogram: số lát cắt theo tháng (study_date)

agg_date = {
    "size": 0,
    "aggs": {
        "by_month": {
            "date_histogram": {
                "field": "study_date",
                "calendar_interval": "month"
            }
        }
    }
}

resp_agg_date = es.search(index=VECTOR_INDEX, body=agg_date)
print("\n=== Số lát cắt theo tháng (study_date) ===")
for b in resp_agg_date["aggregations"]["by_month"]["buckets"]:
    print(b["key_as_string"], "=>", b["doc_count"], "slices")

### 2.5 Profile queries – nhìn bên trong engine <a id="section-2-5"></a>
Thêm `"profile": true` vào request body:

- Elasticsearch trả về chi tiết execution per shard:
  - Các phase: rewrite, weight, scoring.
  - Thời gian từng phase (`time_in_nanos`).
- Rất hữu ích để:
  - So sánh hiệu năng giữa các query.
  - Tối ưu mapping, index, cache, cấu trúc query.

In [ ]:
profile_match_query={
    "profile": True,
    "query": {
        "match": {
            "clinicians_notes": "spondylolisthesis"
        }
    }
}

resp_profile = es.search(index=TEXT_INDEX, body=profile_match_query)

print("Tổng hits:", resp_profile["hits"]["total"]["value"])

for shard in resp_profile.get("profile", {}).get("shards", []):
    print("\n=== Shard", shard.get("id"), "===")
    for search_phase in shard.get("searches", []):
        for q in search_phase.get("query", []):
            print("Query type:", q.get("type"))
            print("  time_in_nanos:", q.get("time_in_nanos"))
            print("  breakdown:", q.get("breakdown"))

### 2.6 Script score <a id="section-2-6"></a>
`function_score` + `script_score` cho phép:

- Tự định nghĩa công thức tính điểm:
  - kết hợp `_score` với field numeric (vd severity, age…),
  - hoặc học từ model ngoài (vd score từ re-ranking model).
- Ở đây dataset text chưa có nhiều numeric field,
  nên demo đơn giản: nhân `_score` với một hệ số.

Trong thực tế, có thể dùng:
- `doc['some_numeric'].value`,
- hoặc vector similarity + `_score`.

In [ ]:
script_score_body = {
    "query": {
        "function_score": {
            "query": {
                "match": {
                    "clinicians_notes": "surgery"
                }
            },
            "script_score": {
                "script": {
                    # ví dụ đơn giản: scale _score
                    "source": "_score * 1.5"
                }
            }
        }
    },
    "size": 5
}

resp_script = es.search(index=TEXT_INDEX, body=script_score_body)
print("=== Script score (match 'surgery') ===")
print_text_hits(resp_script)


### 2.7 Vector kNN search trên `image_vector`
Ở Part 1 ta đã:

- Trích xuất embedding từ BiomedCLIP cho từng lát cắt MRI.
- Lưu vào field `image_vector` (kiểu `dense_vector`, `similarity="cosine"`).

Giờ ta sẽ:

1. Lấy một lát cắt làm **query image**.
2. Dùng vector của lát đó làm `query_vector`.
3. Gọi `knn` search để tìm các lát gần nhất trong embedding space.

In [ ]:
# 1) Lấy một document làm query vector

seed_resp = es.search(
    index=VECTOR_INDEX,
    query={"match_all": {}},
    size=1,
    _source=["patient_id", "body_part_examined", "slice_index", "image_path", "image_vector"],
)

if not seed_resp["hits"]["hits"]:
    raise RuntimeError("VECTOR_INDEX không có data, hãy đảm bảo đã chạy ingest ở Part 1.")

seed_hit = seed_resp["hits"]["hits"][0]
query_vec = seed_hit["_source"]["image_vector"]

print("=== Seed document ===")
print_vector_hits(seed_resp, max_hits=1)

# 2) kNN search

knn_body = {
    "knn": {
        "field": "image_vector",
        "query_vector": query_vec,
        "k": 5,
        "num_candidates": 100
    },
    "_source": ["patient_id", "body_part_examined", "slice_index", "image_path"],
}

resp_knn = es.search(index=VECTOR_INDEX, body=knn_body)

print("\n=== Top 5 nearest neighbours (image_vector) ===")
print_vector_hits(resp_knn, max_hits=5)


### 2.8 Semantic search trên `clinicians_notes` <a id="section-2-8"></a>

Ngoài BM25 (`match`/`match_phrase`), ta có thể:

- Embed mỗi `clinicians_notes` thành vector (vd `all-MiniLM-L6-v2`).
- Lưu vào field `report_embedding` (dense_vector) trong `radiology_text`.
- Chạy `knn` theo nghĩa (semantic) thay vì chỉ theo keyword.

Phần này:

1. Thêm mapping cho `report_embedding`.
2. Dùng SentenceTransformer để embed tất cả notes.
3. Viết hàm `semantic_search(question, k)` → chạy kNN.


In [ ]:
# 1) Thêm field dense_vector cho semantic embedding (nếu chưa có)
es.indices.put_mapping(
    index=TEXT_INDEX,
    properties={
        "report_embedding": {
            "type": "dense_vector",
            "dims": 384,
            "index": True,
            "similarity": "cosine",
        }
    },
)

device_st = "cuda" if torch.cuda.is_available() else "cpu"
print("SentenceTransformer device:", device_st)

st_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2").to(device_st)

@torch.inference_mode()
def embed_text(text: str) -> list[float]:
    # encode trả về numpy array (384-dims)
    vec = st_model.encode([text], convert_to_numpy=True)[0]
    return vec.tolist()


In [ ]:
# 2) Lấy tất cả docs và update report_embedding (cẩn thận size; dataset này nhỏ nên OK)

resp_all = es.search(
    index=TEXT_INDEX,
    query={"match_all": {}},
    size=10_000,
)

hits = resp_all["hits"]["hits"]
print(f"Computing embeddings for {len(hits)} documents...")

ops = []
for h in hits:
    note = h["_source"].get("clinicians_notes", "")
    vec = embed_text(note)
    ops.append(
        {
            "_op_type": "update",
            "_index": TEXT_INDEX,
            "_id": h["_id"],
            "doc": {
                "report_embedding": vec,
            },
        }
    )

helpers.bulk(es, ops)
es.indices.refresh(index=TEXT_INDEX)
print("Updated all documents with report_embedding")


In [ ]:
def semantic_search(question: str, k: int = 5):
    """
    Semantic search trên clinicians_notes dùng report_embedding + kNN.
    """
    query_vec = embed_text(question)
    body = {
        "knn": {
            "field": "report_embedding",
            "query_vector": query_vec,
            "k": k,
            "num_candidates": 100,
        },
        "_source": ["patient_id", "clinicians_notes"],
    }
    resp = es.search(index=TEXT_INDEX, body=body)
    print(f"\n=== Semantic search ===")
    print(f"Question: {question}")
    print_text_hits(resp, max_hits=k)
    return resp


# Một vài ví dụ semantic search

semantic_search("Which patients have a disc bulge at L4-L5?", k=5)
semantic_search("Which reports mention compression of the thecal sac or nerve roots?", k=5)


## 3. Transaction & Concurrency Control <a id="section-3"></a>

### 3.1 Transaction model: single-document atomicity <a id="section-3-1"></a>

Elasticsearch **không phải** là hệ quản trị quan hệ ACID multi-document,
nhưng vẫn có những guarantee quan trọng:

- Mỗi **document write (index/update/delete)** là một **transaction đơn lẻ**:
  - Ghi hoặc **thành công toàn bộ**, hoặc **thất bại** – không có trạng thái "nửa vời".
- Khi một doc được acknowledge thành công:
  - Thay đổi đã được ghi vào **translog** trên primary shard (và replica tuỳ setting).
  - Nếu node crash:
    - translog được replay khi shard khởi động lại → đảm bảo **durability**.

Tuy nhiên:

- Elasticsearch **không hỗ trợ transaction ACID nhiều document** với commit/rollback như RDBMS.
- Thay vào đó:
  - Mỗi doc write là atomic.
  - Consistency ở mức cluster dựa vào **quorum replication** (primary + replicas).
  - Search hoạt động ở chế độ **near-real-time** (NRT), phụ thuộc vào `refresh_interval`.


### 3.2 Translog, refresh & flush <a id="section-3-2"></a>

Khi index/update một doc:

1. Request đến **primary shard**:
   - Ghi thay đổi vào **in-memory buffer**.
   - Append một bản ghi vào **transaction log (translog)** trên disk.
2. Khi translog được fsync (theo chính sách), write được coi là **durable**.
3. **Refresh**:
   - Định kỳ (mặc định `index.refresh_interval ~ 1s`), Elasticsearch:
     - dựng một **segment** mới từ in-memory buffer,
     - mở segment đó cho search.
   - Kết quả: doc có thể search được sau một khoảng trễ nhỏ → gọi là **near-real-time**.
4. **Flush**:
   - Xoá bớt translog cũ sau khi data đã được ghi xuống segment và an toàn.
   - Liên quan tới quản lý disk & durability lâu dài.
   - Không ảnh hưởng trực tiếp tới việc “doc đã search được hay chưa”.

> TL;DR:  
> - **Durability**: nhờ translog được fsync.  
> - **Search visibility**: nhờ refresh → segment mới được mở cho search.


### 3.3 Bulk & `update_by_query` <a id="section-3-3"></a>
- `bulk` API cho phép:
  - index/update/delete **nhiều document trong 1 request**,
  - nhưng mỗi operation trong bulk là **độc lập**:
    - một số doc có thể thành công,
    - một số doc có thể lỗi (mapping error, version conflict, …),
    - không có rollback toàn bộ như transaction SQL.

- `update_by_query`:
  - chạy một truy vấn (vd `range` hoặc `terms`) để tìm tập doc,
  - apply script update trên từng doc.
  - Nếu một doc bị lỗi (vd script lỗi, bad ID, conflict…):
    - task sẽ báo lỗi cho từng doc,
    - phần còn lại vẫn có thể update thành công.

Điều này nghĩa là:

- Cần xử lý lỗi **từng document**:
  - log chi tiết,
  - có thể retry từng doc riêng lẻ,
  - không kỳ vọng “all-or-nothing” cho cả batch.

Dưới đây là một ví dụ nhỏ để minh hoạ update nhiều patient theo `patient_id` bằng `update_by_query`.


In [ ]:
# Cập nhật một nhóm patient_id bằng update_by_query
update_ids = [str(i) for i in range(2, 10)]  # ví dụ: patient_id từ 2 -> 9

# Query chọn các doc cần update
range_query = {
    "query": {
        "ids": {
            "values": update_ids
        }
    }
}

print("=== Trước update_by_query ===")
resp_before = es.search(index=TEXT_INDEX, body={**range_query, "size": 20})
print("Hits:", resp_before["hits"]["total"]["value"])

# Script đơn giản: gắn thêm flag vào notes
script_source = """
if (ctx._source.containsKey("clinicians_notes")) {
    ctx._source.clinicians_notes = ctx._source.clinicians_notes + "[batch updated]";
}
"""
body = {
    "script": {
        "lang": "painless",
        "source": script_source,
    },
    "query": range_query["query"],
}

resp_task = es.update_by_query(
    index=TEXT_INDEX,
    body=body,
    conflicts="proceed",  # vẫn tiếp tục nếu có conflict trên một số doc
    refresh=True,
)

print("\n=== Kết quả update_by_query ===")
print(json.dumps(resp_task.body, indent=2))

print("\n=== Sau update_by_query ===")
resp_after = es.search(index=TEXT_INDEX, body={**range_query, "size": 20})
print("Hits:", resp_after["hits"]["total"]["value"])


### 3.4 Optimistic concurrency control (OCC) <a id="section-3-4"></a>

Vấn đề: **lost update**

- Hai client (A, B) cùng đọc một doc (phiên bản V1).
- A update → doc thành V2.
- B (vẫn nghĩ doc đang là V1) update lên → doc thành V3, *đè* mất update của A.
- Ta muốn phát hiện tình huống này, thay vì âm thầm last-write-wins.

Elasticsearch dùng **optimistic concurrency control**:

- Mỗi document có:
  - `_seq_no` (sequence number – tăng dần mỗi lần write),
  - `_primary_term` (gắn với vòng đời primary shard).
- Khi update:

  - Client đọc document, kèm `_seq_no` và `_primary_term` hiện tại.
  - Gửi update với các tham số:

    ```text
    if_seq_no=<seq_no_cũ>
    if_primary_term=<primary_term_cũ>
    ```

  - Nếu trong lúc đó một write khác đã cập nhật doc:
    - `_seq_no` thay đổi.
    - Elastic sẽ trả về **409 Conflict**.


In [ ]:
# Chọn một doc trong radiology_text để demo concurrency
TARGET_DOC_ID = None

def pick_target_doc():
    """
    Chọn một document bất kỳ trong TEXT_INDEX làm mục tiêu cho demo concurrency.
    """
    global TARGET_DOC_ID
    resp = es.search(
        index=TEXT_INDEX,
        query={"match_all": {}},
        size=1,
        seq_no_primary_term=True,
    )
    if not resp["hits"]["hits"]:
        raise RuntimeError("TEXT_INDEX không có dữ liệu – hãy chạy ingest ở Part 1 trước.")

    hit = resp["hits"]["hits"][0]
    TARGET_DOC_ID = hit["_id"]

    print("=== Target document for concurrency demo ===")
    print("doc_id      :", TARGET_DOC_ID)
    print("patient_id  :", hit["_source"].get("patient_id"))
    print("seq_no      :", hit.get("_seq_no"))
    print("primary_term:", hit.get("_primary_term"))
    print("clinicians_notes (truncated):")
    notes = hit["_source"].get("clinicians_notes", "")
    print((notes[:300] + "...") if len(notes) > 300 else notes)

pick_target_doc()

In [ ]:
def get_doc_with_version(doc_id: str):
    """
    Lấy doc cùng với _seq_no và _primary_term hiện tại.
    """
    doc = es.get(index=TEXT_INDEX, id=doc_id)
    return {
        "id": doc["_id"],
        "seq_no": doc["_seq_no"],
        "primary_term": doc["_primary_term"],
        "source": doc["_source"],
    }

# test
current = get_doc_with_version(TARGET_DOC_ID)
print("Current seq_no:", current["seq_no"], "primary_term:", current["primary_term"])


In [ ]:
def update_notes_thread(thread_id: int, use_occ: bool = True, sleep_before_update: float = 0.5):
    """
    Mỗi thread đơn lẻ:
    1. Đọc doc + phiên bản (_seq_no, _primary_term).
    2. Chỉnh sửa clinicians_notes.
    3. Gửi update:
       - nếu use_occ=True: dùng if_seq_no & if_primary_term (OCC).
       - nếu use_occ=False: update bình thường (last write wins).
    """
    # 1) đọc phiên bản hiện tại
    doc_info = get_doc_with_version(TARGET_DOC_ID)
    seq_no = doc_info["seq_no"]
    primary_term = doc_info["primary_term"]
    notes = doc_info["source"].get("clinicians_notes", "")

    new_notes = notes + f"\n[update from thread {thread_id}]"

    # ngủ một chút để tăng khả năng 2 thread "đụng nhau"
    time.sleep(sleep_before_update)

    try:
        if use_occ:
            # Optimistic concurrency: fail nếu seq_no/primary_term đã khác
            es.update(
                index=TEXT_INDEX,
                id=TARGET_DOC_ID,
                if_seq_no=seq_no,
                if_primary_term=primary_term,
                doc={"clinicians_notes": new_notes},
            )
        else:
            # Không dùng OCC: last write wins
            es.update(
                index=TEXT_INDEX,
                id=TARGET_DOC_ID,
                doc={"clinicians_notes": new_notes},
            )
        print(f"Thread {thread_id}: update thành công (use_occ={use_occ})")
    except ConflictError as e:
        print(f"Thread {thread_id}:  Conflict 409 (use_occ={use_occ}) - {e.info}")
    except Exception as e:
        print(f"Thread {thread_id}: Lỗi khác - {e}")


In [ ]:
def run_concurrent_updates(use_occ: bool = True, rounds: int = 3):
    print(f"\n=== Concurrent updates (use_occ={use_occ}) ===\n")
    for i in range(rounds):
        print(f"--- Round {i+1} ---")

        # đọc doc trước khi update
        before = get_doc_with_version(TARGET_DOC_ID)
        print("Before:")
        print("  seq_no      :", before["seq_no"])
        print("  primary_term:", before["primary_term"])
        print("  tail notes  :", (before["source"]["clinicians_notes"][-120:]))

        # tạo 2 thread cùng update cùng doc
        t1 = threading.Thread(target=update_notes_thread, args=(1, use_occ, 0.5))
        t2 = threading.Thread(target=update_notes_thread, args=(2, use_occ, 0.5))

        t1.start()
        t2.start()
        t1.join()
        t2.join()

        # đọc lại doc
        after = get_doc_with_version(TARGET_DOC_ID)
        print("After:")
        print("  seq_no      :", after["seq_no"])
        print("  primary_term:", after["primary_term"])
        print("  tail notes  :", (after["source"]["clinicians_notes"][-120:]))
        print()

# 1) Demo với OCC: mong đợi 1 thread thành công, 1 thread conflict
run_concurrent_updates(use_occ=True, rounds=3)


In [ ]:
# 2) Demo không dùng OCC: last write wins, có nguy cơ lost update
run_concurrent_updates(use_occ=False, rounds=3)

## 4. Comparison with Other Databases <a id="section-4"></a>

### 4.1. So sánh kỹ thuật tổng quan <a id="section-4-1"></a>
| Tiêu chí                      | Elasticsearch                                                | PostgreSQL                                                                 |
|------------------------------|--------------------------------------------------------------|----------------------------------------------------------------------------|
| Mục tiêu thiết kế            | Search & analytics engine, scale-out, near-real-time search | Hệ quản trị CSDL quan hệ, ưu tiên transaction, nhất quán, OLTP            |
| Data model                   | Document JSON, schema linh hoạt (mapping)                   | Bảng, hàng, cột, schema chặt, nhiều ràng buộc (PK/FK/UNIQUE/CHECK…)       |
| Lược đồ (schema)             | Mapping dynamic, dễ add field mới                           | `CREATE TABLE`, thay đổi schema cần `ALTER TABLE`, được kiểm soát chặt     |
| Full-text search             | Inverted index + analyzer, Query DSL (`match`, `bool`, aggs)| `tsvector`/`tsquery` + GIN index, hoặc `pg_trgm` cho similarity search    |
| Vector search                | `dense_vector` + kNN (HNSW)                                 | Extension `vector` (pgvector) + `ivfflat` / HNSW trên kiểu `vector`       |
| Ngôn ngữ truy vấn            | JSON Query DSL                                              | SQL giàu biểu đạt, joins, subquery, window functions                      |
| Transaction & consistency    | Single-doc atomic, quorum replication, NRT search           | ACID multi-row/multi-table, rất hợp cho hệ thống nghiệp vụ cốt lõi        |
| Concurrency                  | Optimistic concurrency bằng `_seq_no` / `_primary_term`     | MVCC, transaction isolation levels                                        |
| Scalability                  | Shard, replica, scale-out ngang                             | Scale-up chính, partitioning / sharding có nhưng phức tạp hơn             |
| Use case điển hình           | Log, metric, search UI, analytics dashboard, observability  | OLTP, billing, HIS/EMR core data, transaction tài chính, báo cáo chuẩn    |

In [23]:
def get_pg_conn():
    return psycopg2.connect(
        host=PG_HOST,
        port=PG_PORT,
        dbname=PG_DB,
        user=PG_USER,
        password=PG_PASSWORD,
    )


def percentile(values, q):
    arr = np.array(values, dtype=float)
    return float(np.percentile(arr, q))


def summarize_timings(label, timings_ms):
    return {
        "workload": label,
        "runs": len(timings_ms),
        "p50_ms": percentile(timings_ms, 50),
        "p95_ms": percentile(timings_ms, 95),
        "mean_ms": float(np.mean(timings_ms)),
    }

### 4.2. Schema PostgreSQL cho benchmark <a id="section-4-2"></a>

In [24]:
def setup_postgres_schema():
    """Setup PostgreSQL tables, indexes, and extensions"""
    conn = get_pg_conn()
    cur = conn.cursor()
    
    # Extensions
    cur.execute("CREATE EXTENSION IF NOT EXISTS pg_trgm;")
    cur.execute("CREATE EXTENSION IF NOT EXISTS vector;")
    
    # Bảng text reports 
    cur.execute("""
        DROP TABLE IF EXISTS pg_radiology_reports CASCADE;
        CREATE TABLE pg_radiology_reports (
            id BIGSERIAL PRIMARY KEY,
            patient_id TEXT,
            clinicians_notes TEXT NOT NULL,
            note_tsv tsvector GENERATED ALWAYS AS (
                to_tsvector('english', coalesce(clinicians_notes, '')) 
            ) STORED
        );
        CREATE INDEX idx_pg_reports_tsv_gin
            ON pg_radiology_reports USING GIN (note_tsv);
        CREATE INDEX idx_pg_reports_trgm_gin
          ON pg_radiology_reports USING GIN (clinicians_notes gin_trgm_ops);
    """)
    
    # Bảng image vectors
    cur.execute("""
        DROP TABLE IF EXISTS pg_mri_images CASCADE;
        CREATE TABLE pg_mri_images (
            image_id     TEXT PRIMARY KEY,
            patient_id   TEXT,
            body_part    TEXT,
            tags         TEXT[],
            image_vector vector(512) 
        );
        CREATE INDEX idx_pg_mri_tags_gin
          ON pg_mri_images USING GIN (tags);
        CREATE INDEX idx_pg_mri_vec_ivfflat
          ON pg_mri_images USING ivfflat (image_vector vector_l2_ops)
          WITH (lists = 100);
    """)
    
    conn.commit()
    cur.close()
    conn.close()
    print("PostgreSQL schema created!")

setup_postgres_schema()

OperationalError: connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "username"
connection to server at "localhost" (127.0.0.1), port 5432 failed: FATAL:  password authentication failed for user "username"


### 4.3. Ingest to PostgreSQL

#### 4.3.1. Ingest text data

In [ ]:
def fetch_reports_from_es(batch_size=500):
    """
    Stream toàn bộ doc từ index TEXT_INDEX (radiology_text)
    để đổ sang PostgreSQL.
    """
    query = {"query": {"match_all": {}}}
    for hit in scan(
        es,
        index=TEXT_INDEX,
        query=query,
        size=batch_size,
        _source=["patient_id", "clinicians_notes"],
    ):
        src = hit["_source"]
        yield (
            src.get("patient_id"),
            src.get("clinicians_notes", ""),
        )


def load_reports_into_postgres():
    conn = get_pg_conn()
    cur = conn.cursor()

    rows = list(fetch_reports_from_es())
    print(f"Copying {len(rows)} text docs from ES -> PostgreSQL...")

    execute_batch(
        cur,
        """
        INSERT INTO pg_radiology_reports (patient_id, clinicians_notes)
        VALUES (%s, %s);
        """,
        rows,
        page_size=1000,
    )

    conn.commit()
    cur.close()
    conn.close()
    print("✅ Done loading reports into pg_radiology_reports.")


load_reports_into_postgres()

#### 4.3.2. Ingest image data

In [ ]:
def vec_to_pg_str(vec):
    """
    Chuyển Python list[float] -> string dạng '[0.1, 0.2, ...]'
    mà extension vector hiểu được.
    """
    return "[" + ",".join(f"{float(x):.6f}" for x in vec) + "]"


def fetch_images_from_es(batch_size=500):
    """
    Stream doc từ index VECTOR_INDEX (radiology_vectors)
    để đổ sang PostgreSQL.
    """
    fields = ["patient_id", "body_part_examined", "tags", "image_vector"]
    query = {"query": {"match_all": {}}}
    for hit in scan(
        es,
        index=VECTOR_INDEX,
        query=query,
        size=batch_size,
        _source=fields,
    ):
        src = hit["_source"]
        image_id = hit["_id"]
        patient_id = src.get("patient_id")
        body_part = src.get("body_part_examined")
        tags = src.get("tags", [])
        vec = src.get("image_vector")
        if not vec:
            continue
        yield (
            image_id,
            patient_id,
            body_part,
            tags,
            vec_to_pg_str(vec),
        )


def load_images_into_postgres():
    conn = get_pg_conn()
    cur = conn.cursor()

    batch = []
    count = 0

    for row in fetch_images_from_es():
        batch.append(row)
        if len(batch) >= 1000:
            execute_batch(
                cur,
                """
                INSERT INTO pg_mri_images (image_id, patient_id, body_part, tags, image_vector)
                VALUES (%s, %s, %s, %s, %s::vector)
                ON CONFLICT (image_id) DO NOTHING;
                """,
                batch,
            )
            count += len(batch)
            batch = []

    if batch:
        execute_batch(
            cur,
            """
            INSERT INTO pg_mri_images (image_id, patient_id, body_part, tags, image_vector)
            VALUES (%s, %s, %s, %s, %s::vector)
            ON CONFLICT (image_id) DO NOTHING;
            """,
            batch,
        )
        count += len(batch)

    conn.commit()
    cur.close()
    conn.close()
    print(f"✅ Done loading {count} image vectors into pg_mri_images.")


load_images_into_postgres()


### 4.4. Workloads benchmark

#### 4.4.1. **Top-k text search** 
- Elasticsearch: `match` trên `clinicians_notes`.
- PostgreSQL: `note_tsv @@ plainto_tsquery('english', q)` + `ts_rank_cd`.

In [ ]:
TEXT_QUERY = "disc bulge at L4-L5" 


def benchmark_text_es(query=TEXT_QUERY, k=10, loops=30):
    timings = []
    for i in range(loops):
        t0 = time.perf_counter()
        es.search(
            index=TEXT_INDEX,
            query={"match": {"clinicians_notes": query}},
            size=k,
        )
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)  # ms
    return summarize_timings("text_search_es", timings)


def benchmark_text_pg(query=TEXT_QUERY, k=10, loops=30):
    timings = []
    conn = get_pg_conn()
    cur = conn.cursor()

    for i in range(loops):
        t0 = time.perf_counter()
        cur.execute(
            """
            SELECT id, patient_id,
                   ts_rank_cd(note_tsv, plainto_tsquery('english', %s)) AS rank
            FROM pg_radiology_reports
            WHERE note_tsv @@ plainto_tsquery('english', %s)
            ORDER BY rank DESC
            LIMIT %s;
            """,
            (query, query, k),
        )
        cur.fetchall()
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)

    cur.close()
    conn.close()
    return summarize_timings("text_search_pg", timings)


print(benchmark_text_es())
print(benchmark_text_pg())


#### 4.4.2. **Tag filter + aggregation**
- Elasticsearch: filter theo `body_part_examined` + `terms` agg trên `tags`.
- PostgreSQL: `WHERE body_part = ?` + `unnest(tags)` + `GROUP BY`.

In [ ]:
DEFAULT_BODY_PART = "lumbar spine"


def benchmark_tag_agg_es(body_part=DEFAULT_BODY_PART, loops=30):
    timings = []
    body = {
        "query": {
            "bool": {
                "filter": [
                    {"term": {"body_part_examined": body_part}}
                ]
            }
        },
        "aggs": {
            "by_tag": {
                "terms": {"field": "tags", "size": 10}
            }
        },
        "size": 0,
    }

    for i in range(loops):
        t0 = time.perf_counter()
        es.search(index=VECTOR_INDEX, body=body)
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)

    return summarize_timings("tag_filter_agg_es", timings)


def benchmark_tag_agg_pg(body_part=DEFAULT_BODY_PART, loops=30):
    """
    Filter body_part, sau đó đếm tần suất từng tag.
    """
    timings = []
    conn = get_pg_conn()
    cur = conn.cursor()

    for i in range(loops):
        t0 = time.perf_counter()
        cur.execute(
            """
            SELECT tag, COUNT(*) AS cnt
            FROM (
                SELECT unnest(tags) AS tag
                FROM pg_mri_images
                WHERE body_part = %s
            ) AS t
            GROUP BY tag
            ORDER BY cnt DESC
            LIMIT 10;
            """,
            (body_part,),
        )
        cur.fetchall()
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)

    cur.close()
    conn.close()
    return summarize_timings("tag_filter_agg_pg", timings)


print(benchmark_tag_agg_es())
print(benchmark_tag_agg_pg())


#### 4.4.3. **Image vector kNN** 
- Elasticsearch: `knn` trên field `image_vector`.
- PostgreSQL: `ORDER BY image_vector <-> query_vec LIMIT k` (pgvector).

In [ ]:
# Chọn 1 ảnh bất kỳ làm query (query-by-example)
seed_resp = es.search(
    index=VECTOR_INDEX,
    query={"match_all": {}},
    size=1,
    _source=["image_vector", "patient_id", "body_part_examined"],
)

if not seed_resp["hits"]["hits"]:
    raise RuntimeError("VECTOR_INDEX trống – cần ingest dữ liệu ở Part 1.")

seed_hit = seed_resp["hits"]["hits"][0]
QUERY_VEC = seed_hit["_source"]["image_vector"]

print("Seed doc for kNN:")
print("  patient_id  =", seed_hit["_source"].get("patient_id"))
print("  body_part   =", seed_hit["_source"].get("body_part_examined"))


def benchmark_knn_es(query_vec=QUERY_VEC, k=10, loops=30, num_candidates=100):
    timings = []
    body = {
        "knn": {
            "field": "image_vector",
            "query_vector": query_vec,
            "k": k,
            "num_candidates": num_candidates,
        },
        "_source": False,
    }
    for i in range(loops):
        t0 = time.perf_counter()
        es.search(index=VECTOR_INDEX, body=body)
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)
    return summarize_timings("image_knn_es", timings)


def benchmark_knn_pg(query_vec=QUERY_VEC, k=10, loops=30):
    """
    kNN trên pg_mri_images dùng pgvector (L2 distance).
    """
    timings = []
    conn = get_pg_conn()
    cur = conn.cursor()

    vec_str = "[" + ",".join(f"{float(x):.6f}" for x in query_vec) + "]"

    for i in range(loops):
        t0 = time.perf_counter()
        cur.execute(
            """
            SELECT image_id, patient_id, body_part,
                   image_vector <-> %s::vector AS dist
            FROM pg_mri_images
            ORDER BY image_vector <-> %s::vector
            LIMIT %s;
            """,
            (vec_str, vec_str, k),
        )
        cur.fetchall()
        t1 = time.perf_counter()
        timings.append((t1 - t0) * 1000.0)

    cur.close()
    conn.close()
    return summarize_timings("image_knn_pg", timings)


print(benchmark_knn_es())
print(benchmark_knn_pg())


In [ ]:
def run_all_benchmarks():
    results = []

    # Warmup nhẹ để giảm cold-start effect
    benchmark_text_es(loops=3)
    benchmark_text_pg(loops=3)
    benchmark_tag_agg_es(loops=3)
    benchmark_tag_agg_pg(loops=3)
    benchmark_knn_es(loops=3)
    benchmark_knn_pg(loops=3)

    # Run chính
    results.append(benchmark_text_es(loops=20))
    results.append(benchmark_text_pg(loops=20))
    results.append(benchmark_tag_agg_es(loops=20))
    results.append(benchmark_tag_agg_pg(loops=20))
    results.append(benchmark_knn_es(loops=20))
    results.append(benchmark_knn_pg(loops=20))

    df = pd.DataFrame(results)
    display(df)
    return df


benchmark_df = run_all_benchmarks()


In [ ]:
es.close()
print("Elasticsearch connection closed")

### 4.5 Kết luận benchmark (định tính)

Với dataset nhỏ/vừa trong project này (vài trăm–vài nghìn reports và ~1000 lát MRI),
cộng thêm môi trường:

- Elasticsearch chạy single-node, 1 shard.
- Overhead HTTP + JSON cho mỗi request ES.
- PostgreSQL dùng binary wire protocol, query plan được cache tốt.

**Kết quả thực nghiệm (trên máy lab)** thường có dạng:

- PostgreSQL có p50/p95 latency **nhỏ hơn một chút** so với Elasticsearch cho:
  - Top-k text search (FTS `tsvector` + GIN).
  - Vector kNN search trên `pg_mri_images` (pgvector).

Tuy nhiên:

- Benchmark này **không phủ định** Elasticsearch:
  - Dataset còn nhỏ, scale chưa chạm ngưỡng Elasticsearch phát huy sức mạnh.
  - ES mạnh hơn rõ rệt khi:
    - Lượng document lên hàng chục triệu,
    - Cần aggregations phức tạp, dashboards, multi-tenant search,
    - Cần scale-out nhiều node.

### 4.6 Vai trò đề xuất

- **PostgreSQL**:
  - Làm **source of truth** cho dữ liệu bệnh nhân, kết quả xét nghiệm, billing, lịch sử điều trị.
  - Dùng FTS (`tsvector`, `pg_trgm`) + `pgvector` cho những hệ thống nhỏ / đơn node
    cần vừa transaction, vừa search cơ bản.

- **Elasticsearch**:
  - Làm **search layer** phía trước:
    - Tìm kiếm full-text nhanh, highlight, suggest.
    - Vector search cho ảnh tương tự (MRI / X-ray / báo cáo embedding).
    - Aggregations real-time cho dashboards & analytics.
  - Kết hợp tốt với log/metric, observability, audit trail.

Một kiến trúc hợp lý:

- PostgreSQL: lưu dữ liệu chuẩn hóa, đảm bảo tính nhất quán & audit.
- Một ETL / replication pipeline:
  - đọc từ Postgres (hoặc từ hệ thống DICOM),
  - đẩy sang Elasticsearch (text reports + image vectors),
  - phục vụ các UI search / analytics.